In [37]:
from collections import deque, namedtuple


DIRS = {
    'N': (0, 1),
    'S': (0, -1),
    'E': (1, 0),
    'W': (-1, 0),
}


class Node:
    x = None
    y = None
    neighbors = None
    grid = None

    def __init__(self, x, y, grid):
        self.x = x
        self.y = y
        self.grid = grid
        grid[(x, y)] = self
        self.neighbors = {}
        
    def move(self, d):
        if d in self.neighbors:
            return self.neighbors[d]
        coords = (self.x + d[0], self.y + d[1])
        if coords not in self.grid:
            Node(coords[0], coords[1], self.grid)
        node = self.grid[coords]
        self.neighbors[d] = node
        node.neighbors[(-d[0], -d[1])] = self        
        return node

        
#^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$
class Mapper:
    root = None
    grid = None

    def __init__(self, filename=None, text=None):
        self.grid = {}
        if not text:
            with open(filename) as infile:
                text = infile.readline().strip()
        text = list(text[:0:-1])
        self.root = Node(0, 0, self.grid)
        self.parse(text, [self.root])

    def parse(self, text, fringe):
        while text:
            c = text[-1]
            if c in ')$|':
                return fringe
            elif c in DIRS:
                fringe = [n.move(DIRS[c]) for n in fringe]
                text.pop()
                continue
            elif c == '(':
                text.pop()
                new_fringe = []
                while c != ')':
                    new_fringe.extend(self.parse(text, fringe))
                    c = text.pop()
                fringe = list(set(new_fringe))
            else:
                raise ValueError('Unrecognized input: {}'.format(c))

    def traverse(self):
        fringe = {self.root}
        doors = -1
        far_rooms = 0
        seen = {self.root}
        while fringe:
            doors += 1
            if doors >= 1000:
                far_rooms += len(fringe)
            new_fringe = set()
            for node in fringe:
                [new_fringe.add(neigh) for neigh in node.neighbors.values() if neigh not in seen]
            seen |= new_fringe
            fringe = new_fringe
            
        return doors, far_rooms
        

In [38]:
m = Mapper(text='^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$')
m.traverse()

(18, 0)

In [39]:
m = Mapper(filename='input.txt')
m.traverse()

(3656, 8430)